#### Step6: Find Hashtags, Named Entities, Text with emojis etc from extracted text for text cues

In [32]:
file_name = 'step5_data.pkl'
df = pd.read_pickle(file_name)

In [33]:
import preprocessor as tweet_p
import re

In [34]:
tweet_p.set_options(tweet_p.OPT.URL,tweet_p.OPT.MENTION)

In [35]:
from emoji import UNICODE_EMOJI

In [5]:
def is_emoji(s):
    return s in UNICODE_EMOJI


In [6]:
from nltk.tokenize import word_tokenize

In [7]:
import spacy    
nlp = spacy.load("en_core_web_sm")
all_stopwords = set(nlp.Defaults.stop_words)
#print(set(all_stopwords))

In [8]:
#


In [9]:
orig_line_str = "What a great way to connect cricket with learning! I think I learnt a few new concepts today!"

In [10]:
cues_1 = set()

In [11]:
def gather_details(orig_line_str):
    
    orig_line_str = re.sub(r'^RT', '', orig_line_str)

    doc = nlp(orig_line_str) 

    named_entities = set()

    for nc in doc.noun_chunks:
        flag = False
        for w in nlp(str(nc)):
            if w.tag_ == 'NNP':
                named_entities.add(nc.text) 

    orig_line = word_tokenize(orig_line_str)
    exclamation_indices = []
    emoji_indices = []
    emojis = []
    hashtags = []

    temp = []

    #  Coach #Bigil 🔥🔥 Another fav scene
    for w in orig_line:
        #print(w)
        word = w[0]

        for character in range(1,len(w)):
            #print(w[character])
            if w[character] in UNICODE_EMOJI:
                #print(w[character])

                if word[-1] not in UNICODE_EMOJI:
                    word += ' '+w[character].strip()
                else:
                    #print(w[character])
                    if word[-1] == w[character]:
                        pass
                    word = word.strip()+w[character]
            else:
                #print(w[character])
                word = word.strip() + w[character]
            #print(word)

        temp.append(word.strip())


    line_str = ' '.join(temp)
    line_str = re.sub('# ','#',line_str)
    line = word_tokenize(line_str)

    c = 0
    for word_index in range(len(orig_line)):
        if orig_line[word_index] in all_stopwords:
            continue
        elif orig_line[word_index].startswith('#'):
            hashtags.append('#'+orig_line[word_index+1])

        elif orig_line[word_index] == '!':
            exclamation_indices.append(word_index)

        elif is_emoji(orig_line[word_index][0]) :
            emoji_indices.append(word_index)
            if len(orig_line[word_index]) > 1 and len(list(set(orig_line[word_index]))) == 1:
                emojis.append(orig_line[word_index][0])
            else:
                emojis.append(orig_line[word_index])




    return exclamation_indices,emoji_indices, hashtags,named_entities, emojis,orig_line,line,line_str

In [12]:
exclamation_indices,emoji_indices, hashtags,named_entities, emojis,orig_line,line,line_str = gather_details(orig_line_str)

In [13]:
print(line_str)
print(orig_line_str)

What a great way to connect cricket with learning ! I think I learnt a few new concepts today !
What a great way to connect cricket with learning! I think I learnt a few new concepts today!


In [14]:
#a sentence with a hashtag provided there are non hashtag words in sentence. No punctuation marks like full stops limited to 5 words. if sentence is more than 5 words, see if left or right side has more hashtag words within distance of 5. take those words
def hashtag_cues(orig_line_str,hashtags):
    chunks = re.split(',|\.|:|&|\!|\?', orig_line_str)

    cues_1 = set()
    #chunks_set = set(chunks)
    if len(hashtags)>0:

        if len (orig_line_str)>=6:

            for ht in hashtags:
                #print(ht)
                for chunk in chunks:
                    #print(chunk)
                    if ht in chunk:
                        no_stops = ' '.join(w for w in chunk.split() if w not in all_stopwords)
                        if len(no_stops)>5:
                            twos = chunk.split(ht)
                            #print(twos)
                            left = twos[0] + ' ' + ht
                            right = ht + ' ' + twos[1]
                            left_no_stops = [w for w in left.split() if w not in all_stopwords]
                            #print(left_no_stops)
                            try:
                                right_no_stops = [w for w in right.split() if w not in all_stopwords]
                                if len(right_no_stops)>=3:
                                    if len(right_no_stops)>=5:
                                        cues_1.add(' '.join(right.split()[:5]))
                                    else:
                                        cues_1.add(right)
                            except:
                                pass

                            if len(left_no_stops)>=3:
                                if len(left_no_stops)>=5:
                                    cues_1.add(' '.join(left.split()[-5:]))
                                else:
                                    cues_1.add(left)
    return cues_1
 

In [15]:
#print(hashtag_cues(orig_line_str,hashtags))
cues_1 = cues_1.union(hashtag_cues(orig_line_str,hashtags))

In [16]:
cues_1

set()

In [17]:
#if exclamation mark there, take 5 words that lead upto it.

def exclamation_cues(exclamation_indices,orig_line):
    
    cues_1 = set()
    if len(exclamation_indices) != 0:
        actual_indices = [exclamation_indices[0]]
        for i in range(1, len(exclamation_indices)):
            if exclamation_indices[i] == exclamation_indices[i-1]+1:
                continue
            else:
                actual_indices.append(exclamation_indices[i])


        exclamation_cues = []
        for i in actual_indices:
            try:
                exclamation_cues.append(' '.join(orig_line[i-5:i+1]))
            except:
                exclamation_cues.append(' '.join(orig_line[0:i+1]))
        #print(exclamation_cues)
        for j in exclamation_cues:

            chunks_ex = re.split(',|\.|:|&|\!|\?', j)
            #print(chunks_ex)
            if len(chunks_ex)>1:
                cues_1.add(chunks_ex[-2])
    return cues_1

In [18]:
cues_1 = cues_1.union(exclamation_cues(exclamation_indices,orig_line))
    

In [19]:
cues_1

{'a few new concepts today ', 'to connect cricket with learning '}

In [20]:
### if emoji in line but preceeded by fullstop, do ner for line and take 5 words from left/right
def emoji_1(emoji_indices,orig_line,named_entities,orig_line_str):

    cues_1 = set()
    if len(emoji_indices)!= 0:
        for e in emoji_indices:
            try:
                if orig_line[e-1] in {'.','?',',',':','!'}:
                    if len(named_entities) != 0:
                        for ne in named_entities:
                            #print(ne)
                            ne_left = ne.split()[0]
                            ne_right = ne.split()[-1]
                            try:
                                left_cue = orig_line[orig_line.index(ne_left)-4:orig_line.index(ne_left)+1]
                            except:
                                left_cue = orig_line[:orig_line.index(ne_left)+1]
                            try:
                                right_cue = orig_line[orig_line.index(ne_right)+1:orig_line.index(ne_left)+5]
                            except:
                                right_cue = orig_line[orig_line.index(ne_right):]
                            cues_1.add((' '.join(left_cue) + ' ' + ne).strip())
                            cues_1.add((ne + ' ' + ' '.join(right_cue)).strip())
                else:

                    no_stops = [w for w in orig_line_str.split() if w not in all_stopwords]
                    if len(no_stops) <=5:
                        cues_1.add(orig_line_str)
                    else:
                        if len(named_entities) != 0:
                            for ne in named_entities:
                                #print(ne)
                                ne_left = ne.split()[0]
                                ne_right = ne.split()[-1]
                                try:
                                    left_cue = orig_line[orig_line.index(ne_left)-4:orig_line.index(ne_left)]
                                except:
                                    left_cue = orig_line[:orig_line.index(ne_left)+1]
                                try:
                                    right_cue = orig_line[orig_line.index(ne_right)+1:orig_line.index(ne_left)+5]
                                except:
                                    #print(ne_right,orig_line)
                                    right_cue = orig_line[orig_line.index(ne_right):]
                                cues_1.add((' '.join(left_cue) + ' ' + ne).strip())
                                cues_1.add((ne + ' ' + ' '.join(right_cue)).strip())

            except Exception as e:
                #print(e)
                continue        
    return cues_1
                


In [21]:
cues_1 = cues_1.union(emoji_1(emoji_indices,orig_line,named_entities,orig_line_str))

In [22]:
cues_1

{'a few new concepts today ', 'to connect cricket with learning '}

In [23]:
def emoji_2(line_str, emojis):

    chunks = re.split(',|\.|:|&|\!|\?', line_str)

    cues_1 = set()
    #chunks_set = set(chunks)
    if len(emojis)>0:

        if len (line_str)>=6:

            for ej in emojis:
                #print(ej)
                for chunk in chunks:
                    #print(chunk)
                    if ej in chunk:
                        no_stops = ' '.join(w for w in chunk.split() if w not in all_stopwords)
                        if len(no_stops)>5:
                            twos = chunk.split(ej)
                            #print(twos)
                            left = twos[0] + ' ' + ej
                            right = ej + ' ' + twos[1]
                            left_no_stops = [w for w in left.split() if w not in all_stopwords]
                            #print(left_no_stops)
                            try:
                                right_no_stops = [w for w in right.split() if w not in all_stopwords]
                                if len(right_no_stops)>=3:
                                    if len(right_no_stops)>=5:
                                        cues_1.add(' '.join(right.split()[:5]))
                                    else:
                                        cues_1.add(right)
                            except:
                                pass

                            if len(left_no_stops)>=3:
                                if len(left_no_stops)>=5:
                                    cues_1.add(' '.join(left.split()[-5:]))
                                else:
                                    cues_1.add(left)
    return cues_1

In [24]:
cues_1 = cues_1.union(emoji_2(line_str, emojis))

In [25]:
def get_text_cues(orig_line_str):
    
    cues_1 = set()
    exclamation_indices,emoji_indices, hashtags,named_entities, emojis,orig_line,line,line_str = gather_details(orig_line_str)
    cues_1 = cues_1.union(hashtag_cues(orig_line_str,hashtags))
    cues_1 = cues_1.union(exclamation_cues(exclamation_indices,orig_line))
    cues_1 = cues_1.union(emoji_1(emoji_indices,orig_line,named_entities,orig_line_str))
    cues_1 = cues_1.union(emoji_2(line_str, emojis))
    cues_1.discard('RT')
    cues_1.discard('RT :')
    return list(cues_1)

In [26]:
df.head(2)

,title,popularity,Type,Related Topics,regions_of_interest,Tweets,Reddit_Posts,Sentiment_Score
0,#DubbakaBypolls,99.000000,hashtag,NaN,"[[0, West Bengal], [0, Uttarakhand], [0, Uttar...",[RT @Rehanpawanist_: Hope you got an idea of s...,NaN,17.0
1,#IrandamKuththu,77.141566,hashtag,"[Kuththu, Film, Irandam Ulagam, Download]","[[100, Tamil Nadu], [59, Kerala], [19, Karnata...",NaN,NaN,NaN


In [28]:
mapping = {}
for title,tweets,reddits in zip(df['title'],df['Tweets'],df['Reddit_Posts']):
    
    
    try:
        cues = []
        for j in tweets:

            j = j.splitlines()
            for tweet in j:
                tweet = tweet_p.clean(re.sub(r'^RT : ', '', tweet))
                cues += get_text_cues(tweet)
                
        try:
            
            for j in reddits:

                j = j.splitlines()
                for post in j:
                    cues += get_text_cues(post)
        except Exception as e:
            pass
            #print(e)
        
        mapping.update({title:cues})
        print(title)
    except Exception as e:
        pass
        

#DubbakaBypolls
Bihar Assembly
#BiharElectionResults
bighit
Jaipur
Starc
#BoycottAmazon
EVMs
Exit
#Bigil
#BiharResult
Jharkhand
#IPLUnacademyVideo
India Today
Joker
Owaisi
Burnol
NDTV
Mozambique
Rona
Mahagathbandhan
Nitish
Armenia
#BiharElections2020


In [29]:
df['Text_Cues'] = df['title'].map(mapping)

In [30]:
df

,title,popularity,Type,Related Topics,regions_of_interest,Tweets,Reddit_Posts,Sentiment_Score,Text_Cues
0,#DubbakaBypolls,99.000000,hashtag,NaN,"[[0, West Bengal], [0, Uttarakhand], [0, Uttar...",[RT @Rehanpawanist_: Hope you got an idea of s...,NaN,17.000000,"[#DubbakaBypolls • #DubbakaElections, #Dubbak..."
1,#IrandamKuththu,77.141566,hashtag,"[Kuththu, Film, Irandam Ulagam, Download]","[[100, Tamil Nadu], [59, Kerala], [19, Karnata...",NaN,NaN,NaN,NaN
2,Bihar Assembly,63.786451,Topic,"[Kasturbhai Lalbhai, Constituent Assembly of I...","[[100, Bihar], [61, Jharkhand], [48, Tripura],...",[RT @YangerINC: The Bihar Poll story according...,NaN,20.625000,[]
3,NEET results,48.965465,Entrance examination,"[2017, NEET · 2017, NEET · 2019, NEET · 2018]","[[100, Andhra Pradesh], [63, Jammu and Kashmir...",NaN,NaN,NaN,NaN
4,#BiharElectionResults,47.482541,hashtag,"[2017, Amethi, United States presidential elec...","[[100, Bihar], [65, Daman and Diu], [54, Dadra...",[RT @Kattar_Hindu11: And I was thinking that n...,NaN,16.666667,"[now politics will be interesting , बात नहीं ह..."
5,bighit,46.416561,Company,"[GFriend, Seventeen, Concert tour, Blackpink]","[[100, Mizoram], [48, Nagaland], [45, Arunacha...",[RT @kkoobap: watch bighit's family concert ha...,"[1!....2!...3!.... happy Birthday mochi✨🌟\n, Y...",38.636364,"[#BigHit And Superb Announce Launch, Of New Mo..."
6,Shikhar Dhawan,43.898400,Indian cricketer,"[Hardik Pandya, Rishabh Pant, 2019 ICC Cricket...","[[100, Daman and Diu], [88, Jharkhand], [83, G...",NaN,NaN,NaN,NaN
7,Jaipur,42.561668,City in India,"[Sapna Choudhary, Dupatta, Jaipur Pink Panther...","[[100, Rajasthan], [9, Haryana], [8, Delhi], [...",[RT @ajaymaken: A good news from Urban Rajasth...,"[November sunrise in Amer (near Jaipur,India)\...",2.440476,"[good news from Urban Rajasthan , to dedicate ..."
8,Sira,40.754191,City in India,"[Needham, Sye Raa Narasimha Reddy, Somerville,...","[[100, Chandigarh], [81, Karnataka], [58, Punj...",NaN,NaN,NaN,NaN
9,Starc,35.165562,Food,"[2018 Indian Premier League, 2016 Indian Premi...","[[100, Tripura], [88, Goa], [86, Jammu and Kas...",[RT @LMcKirdy7: Fair to say Mitchell Starc was...,[GRAND FINAL | Serral vs ShoWTimE | Game 6 | W...,30.416667,[quick was n't too happy ]


In [31]:
df.to_pickle('step6 data.pkl')